## Imports

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from lazypredict.Supervised import LazyRegressor
from sklearn.utils import all_estimators
from sklearn.base import RegressorMixin

## Define the files to be loaded

In [2]:
# Participant number whose data should be loaded
participant_number = 2

# The version of the questionnaires to load
quest_version = 2

## Loading the pre-processed data

In [3]:
# Complete dataset (Oura data + questionnaire data)
df = pd.read_csv('data/preprocessed/preprocessed_data_' + str(participant_number) + '_v' + str(quest_version) + '.csv')
df = df.drop(labels=['actual_day', 'actual_day.1'], axis=1)

# Oura dataset
df_oura = pd.read_csv('data/preprocessed/preprocessed_sleep_' + str(participant_number) + '_v' + str(quest_version) + '.csv')
df_oura = df_oura.drop(labels=['actual_day'], axis=1)

# Questionnaire dataset
df_quest = pd.read_csv('data/preprocessed/preprocessed_questionnaires_' + str(participant_number) + '_v' + str(quest_version) + '.csv')
df_quest = df_quest.drop(labels=['actual_day'], axis=1)

In [4]:
# Remove these to try and fix the issues raised by LazyRegressor for them 
# example: 
# AdaBoostRegressor model failed to execute
# Found unknown categories ['[0.0, 51.0, 53.0, 53.0, 54.0, 53.0, ...
df = df.drop(labels=['hrv.items', 'heart_rate.items', 'sleep_phase_5_min', 'movement_30_sec'], axis=1)
df_oura = df_oura.drop(labels=['hrv.items', 'heart_rate.items', 'sleep_phase_5_min', 'movement_30_sec'], axis=1)

In [5]:
print(df.columns.tolist())

['0', 'average_breath', 'average_breath_variation', 'average_heart_rate', 'average_hrv', 'awake_time', 'bedtime_end_delta', 'bedtime_end_seconds', 'bedtime_start_delta', 'bedtime_start_seconds', 'contributors.deep_sleep', 'contributors.efficiency', 'contributors.latency', 'contributors.rem_sleep', 'contributors.restfulness', 'contributors.timing', 'contributors.total_sleep', 'deep_sleep_duration', 'efficiency', 'got_ups', 'latency', 'light_sleep_duration', 'lowest_heart_rate', 'lowest_heart_rate_time_offset', 'period', 'readiness.contributors.activity_balance', 'readiness.contributors.body_temperature', 'readiness.contributors.hrv_balance', 'readiness.contributors.previous_day_activity', 'readiness.contributors.previous_night', 'readiness.contributors.recovery_index', 'readiness.contributors.resting_heart_rate', 'readiness.contributors.sleep_balance', 'readiness.score', 'readiness.temperature_deviation', 'readiness.temperature_trend_deviation', 'rem_sleep_duration', 'restless', 'restle

## Prepare regressor list for lazypredict

In [6]:
removed_regressors = [
    "TheilSenRegressor",
    "ARDRegression", 
    "CCA", 
    "IsotonicRegression", 
    "StackingRegressor",
    "MultiOutputRegressor", 
    "MultiTaskElasticNet", 
    "MultiTaskElasticNetCV", 
    "MultiTaskLasso", 
    "MultiTaskLassoCV", 
    "PLSCanonical", 
    "PLSRegression", 
    "RadiusNeighborsRegressor", 
    "RegressorChain", 
    "VotingRegressor", 
]

In [7]:
REGRESSORS = [
    est
    for est in all_estimators()
    if (issubclass(est[1], RegressorMixin) and (est[0] not in removed_regressors))
]

## Run LazyRegressor on datasets

In [8]:
def lazy_regressor(X, y, test_size, random_state, regressor):
    # Create the test and train sets for the given prediction
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = test_size, random_state = random_state)
    
    models, predictions = regressor.fit(X_train, X_test, y_train, y_test)
    return models

In [9]:
# Apply the LazyRegressor on the data
reg = LazyRegressor(verbose=0, ignore_warnings=False, custom_metric=None, regressors = REGRESSORS)

### LazyRegressor for the factors influencing the sleep scores given all the variables

In [10]:
# Separate the indenpendent variables from the dependent variables
y_oura = df['score']
y_sub = df['subjective_sleep_score']
X = df.drop(labels=['score', 'subjective_sleep_score'], axis=1)

#### Oura sleep scores

In [11]:
models_oura = lazy_regressor(X, y_oura, test_size = 0.2, random_state = 2, regressor = reg)

print(models_oura)

'tuple' object has no attribute '__name__'
Invalid Regressor(s)


 70%|███████   | 28/40 [00:01<00:00, 21.91it/s]

LassoLarsIC model failed to execute
You are using LassoLarsIC in the case where the number of samples is smaller than the number of features. In this setting, getting a good estimate for the variance of the noise is not possible. Provide an estimate of the noise variance in the constructor.


 88%|████████▊ | 35/40 [00:01<00:00, 23.53it/s]

RANSACRegressor model failed to execute
`min_samples` may not be larger than number of samples: n_samples = 77.


100%|██████████| 40/40 [00:01<00:00, 23.50it/s]

                               Adjusted R-Squared  R-Squared   RMSE  \
Model                                                                 
Lars                                     40436.08 -129816.88 237.50   
GaussianProcessRegressor                    48.93    -152.87   8.18   
MLPRegressor                                 3.09      -5.72   1.71   
LassoLarsCV                                  2.07      -2.45   1.22   
QuantileRegressor                            1.36      -0.14   0.70   
DummyRegressor                               1.32      -0.02   0.67   
LassoLars                                    1.32      -0.02   0.67   
Lasso                                        1.32      -0.02   0.67   
ElasticNet                                   1.18       0.43   0.50   
KNeighborsRegressor                          1.11       0.65   0.39   
PassiveAggressiveRegressor                   1.08       0.74   0.34   
SGDRegressor                                 1.05       0.84   0.26   
SVR   

#### Subjective sleep scores

In [12]:
models_sub = lazy_regressor(X, y_sub, test_size = 0.2, random_state = 6, regressor = reg)

print(models_sub)

'tuple' object has no attribute '__name__'
Invalid Regressor(s)


 65%|██████▌   | 26/40 [00:03<00:01,  8.39it/s]

LassoLarsIC model failed to execute
You are using LassoLarsIC in the case where the number of samples is smaller than the number of features. In this setting, getting a good estimate for the variance of the noise is not possible. Provide an estimate of the noise variance in the constructor.


100%|██████████| 40/40 [00:03<00:00, 11.75it/s]

RANSACRegressor model failed to execute
`min_samples` may not be larger than number of samples: n_samples = 77.
                                       Adjusted R-Squared  \
Model                                                       
LinearRegression              505257517734733044252672.00   
TransformedTargetRegressor    505257517734733044252672.00   
Lars                                           1971170.40   
GaussianProcessRegressor                            29.18   
MLPRegressor                                         4.43   
LinearSVR                                            2.32   
HuberRegressor                                       2.30   
PassiveAggressiveRegressor                           2.27   
ExtraTreeRegressor                                   2.03   
Ridge                                                1.94   
KernelRidge                                          1.93   
DecisionTreeRegressor                                1.85   
OrthogonalMatchingPursuit         

### LazyRegressor for the factors influencing the sleep scores given the questionnaire variables only

In [13]:
# Separate the indenpendent variables from the dependent variables
X_quest = df_quest.drop(labels=['subjective_sleep_score'], axis=1)

#### Oura sleep score

In [14]:
models_oura = lazy_regressor(X_quest, y_oura, test_size = 0.2, random_state = 2, regressor = reg)

print(models_oura)

'tuple' object has no attribute '__name__'
Invalid Regressor(s)


100%|██████████| 40/40 [00:00<00:00, 40.58it/s]

                                        Adjusted R-Squared  \
Model                                                        
RANSACRegressor               5550509686441437569220608.00   
TransformedTargetRegressor    3945868123691236162797568.00   
LinearRegression              3945868123691236162797568.00   
KernelRidge                                         217.72   
GaussianProcessRegressor                            133.94   
MLPRegressor                                          6.54   
ExtraTreeRegressor                                    4.21   
ExtraTreesRegressor                                   3.52   
LinearSVR                                             3.17   
DecisionTreeRegressor                                 3.04   
PassiveAggressiveRegressor                            2.68   
QuantileRegressor                                     2.55   
OrthogonalMatchingPursuitCV                           2.41   
OrthogonalMatchingPursuit                             2.41   
Lasso   

#### Subjective sleep scores

In [15]:
models_sub = lazy_regressor(X_quest, y_sub, test_size = 0.2, random_state = 6, regressor = reg)

print(models_sub)

'tuple' object has no attribute '__name__'
Invalid Regressor(s)


100%|██████████| 40/40 [00:00<00:00, 43.64it/s]

                                       Adjusted R-Squared  \
Model                                                       
RANSACRegressor               296299723676300022382592.00   
KernelRidge                                        130.76   
GaussianProcessRegressor                            93.10   
PassiveAggressiveRegressor                           6.46   
MLPRegressor                                         5.62   
DecisionTreeRegressor                                4.12   
LinearSVR                                            3.76   
ExtraTreeRegressor                                   3.54   
KNeighborsRegressor                                  3.49   
ExtraTreesRegressor                                  3.42   
SVR                                                  3.26   
HuberRegressor                                       3.26   
LinearRegression                                     3.17   
TransformedTargetRegressor                           3.17   
Ridge                   

### LazyRegressor for the factors influencing the sleep scores given the Oura variables only

In [16]:
# Separate the indenpendent variables from the dependent variables
X_oura = df_oura.drop(labels=['score'], axis=1)

#### Oura sleep score

In [17]:
models_oura = lazy_regressor(X_oura, y_oura, test_size = 0.2, random_state = 2, regressor = reg)

print(models_oura)

'tuple' object has no attribute '__name__'
Invalid Regressor(s)


100%|██████████| 40/40 [00:01<00:00, 32.15it/s]

                               Adjusted R-Squared  R-Squared  RMSE  Time Taken
Model                                                                         
Lars                                       218.44    -342.33 12.21        0.01
GaussianProcessRegressor                    98.34    -152.69  8.17        0.01
MLPRegressor                                 8.39     -10.68  2.25        0.10
QuantileRegressor                            1.72      -0.14  0.70        0.05
DummyRegressor                               1.65      -0.02  0.67        0.02
Lasso                                        1.65      -0.02  0.67        0.01
LassoLars                                    1.65      -0.02  0.67        0.01
ElasticNet                                   1.36       0.43  0.50        0.01
PassiveAggressiveRegressor                   1.32       0.49  0.47        0.01
ExtraTreeRegressor                           1.22       0.66  0.39        0.01
KNeighborsRegressor                          1.21   

#### Subjective sleep scores

In [18]:
models_sub = lazy_regressor(X_oura, y_sub, test_size = 0.2, random_state = 6, regressor = reg)

print(models_sub)

'tuple' object has no attribute '__name__'
Invalid Regressor(s)


100%|██████████| 40/40 [00:01<00:00, 21.40it/s]

                               Adjusted R-Squared  R-Squared  RMSE  Time Taken
Model                                                                         
Lars                                       435.82    -685.55 20.08        0.01
GaussianProcessRegressor                    58.08     -89.12  7.28        0.01
RANSACRegressor                              8.20     -10.37  2.58        0.06
MLPRegressor                                 6.03      -6.94  2.16        0.15
TransformedTargetRegressor                   4.17      -4.01  1.72        0.01
LinearRegression                             4.17      -4.01  1.72        0.01
HuberRegressor                               3.68      -3.24  1.58        0.02
LinearSVR                                    3.07      -2.27  1.39        0.02
Ridge                                        2.81      -1.85  1.29        0.01
KernelRidge                                  2.78      -1.81  1.28        0.01
PassiveAggressiveRegressor                   2.42   

### LazyRegressor for the factors influencing the average HRV given all the variables

In [19]:
# Separate the indenpendent variables from the dependent variables
y_hrv = df['average_hrv']
X_hrv = df.drop(labels=['average_hrv'], axis=1)
X_hrv_quest = df_quest
X_hrv_oura = df_oura.drop(labels=['average_hrv'], axis=1)

In [20]:
# Drop the rows for which the HRV value is not saved
for i in range(len(df['average_hrv'])):
    if pd.isna(df.iloc[i]['average_hrv']):
        y_hrv = y_hrv.drop(i)
        X_hrv = X_hrv.drop(i)
        X_hrv_quest = X_hrv_quest.drop(i)
        X_hrv_oura = X_hrv_oura.drop(i)

In [21]:
models_hrv = lazy_regressor(X_hrv, y_hrv, test_size = 0.2, random_state = 5, regressor = reg)

print(models_hrv)

'tuple' object has no attribute '__name__'
Invalid Regressor(s)


 65%|██████▌   | 26/40 [00:01<00:00, 14.77it/s]

LassoLarsIC model failed to execute
You are using LassoLarsIC in the case where the number of samples is smaller than the number of features. In this setting, getting a good estimate for the variance of the noise is not possible. Provide an estimate of the noise variance in the constructor.


 90%|█████████ | 36/40 [00:01<00:00, 21.54it/s]

RANSACRegressor model failed to execute
`min_samples` may not be larger than number of samples: n_samples = 76.


100%|██████████| 40/40 [00:01<00:00, 20.31it/s]

                               Adjusted R-Squared  R-Squared    RMSE  \
Model                                                                  
Lars                                      2223.83   -7252.43 1618.24   
GaussianProcessRegressor                     5.25     -12.86   70.75   
MLPRegressor                                 1.96      -2.13   33.62   
DummyRegressor                               1.31      -0.01   19.10   
QuantileRegressor                            1.31      -0.01   19.09   
TransformedTargetRegressor                   1.28       0.09   18.17   
LinearRegression                             1.28       0.09   18.17   
NuSVR                                        1.27       0.12   17.87   
HuberRegressor                               1.27       0.14   17.67   
SVR                                          1.26       0.14   17.63   
PassiveAggressiveRegressor                   1.20       0.35   15.32   
KernelRidge                                  1.19       0.39   1

### LazyRegressor for the factors influencing the average HRV given the questionnaire variables only

In [22]:
models_hrv = lazy_regressor(X_hrv_quest, y_hrv, test_size = 0.2, random_state = 5, regressor = reg)

print(models_hrv)

'tuple' object has no attribute '__name__'
Invalid Regressor(s)


100%|██████████| 40/40 [00:01<00:00, 31.82it/s]

                               Adjusted R-Squared  R-Squared  RMSE  Time Taken
Model                                                                         
KernelRidge                                 19.30     -13.45 72.22        0.01
GaussianProcessRegressor                    17.07     -11.69 67.67        0.01
MLPRegressor                                10.96      -6.86 53.27        0.18
ExtraTreeRegressor                           3.02      -0.60 24.01        0.01
LinearSVR                                    2.90      -0.50 23.29        0.01
HuberRegressor                               2.61      -0.27 21.44        0.02
TransformedTargetRegressor                   2.43      -0.13 20.17        0.01
LinearRegression                             2.43      -0.13 20.17        0.01
Ridge                                        2.29      -0.02 19.17        0.01
LassoLars                                    2.28      -0.01 19.10        0.01
DummyRegressor                               2.28   

### LazyRegressor for the factors influencing the average HRV given the Oura variables only

In [23]:
models_hrv = lazy_regressor(X_hrv_oura, y_hrv, test_size = 0.2, random_state = 5, regressor = reg)

print(models_hrv)

'tuple' object has no attribute '__name__'
Invalid Regressor(s)


100%|██████████| 40/40 [00:01<00:00, 23.92it/s]

                               Adjusted R-Squared   R-Squared     RMSE  \
Model                                                                    
Lars                                   1878323.74 -2965771.74 32721.94   
GaussianProcessRegressor                     9.76      -12.84    70.68   
MLPRegressor                                 3.06       -2.25    34.24   
DummyRegressor                               1.64       -0.01    19.10   
QuantileRegressor                            1.64       -0.01    19.09   
NuSVR                                        1.52        0.18    17.20   
SVR                                          1.50        0.22    16.80   
DecisionTreeRegressor                        1.37        0.42    14.52   
LassoLars                                    1.32        0.49    13.55   
GradientBoostingRegressor                    1.31        0.51    13.32   
TransformedTargetRegressor                   1.29        0.55    12.80   
LinearRegression                      

### LazyRegressor for the factors influencing the total sleep duration given all variables

In [24]:
# Separate the indenpendent variables from the dependent variables
# For the total sleep duration, only consider the days for which a person woke up naturally
y_total_sleep = df[df['Naturally'] == 1]['total_sleep_duration']
X_total_sleep = df[df['Naturally'] == 1]
X_total_sleep = X_total_sleep.drop(labels=['total_sleep_duration', 'deep_sleep_duration', 'light_sleep_duration', 'contributors.total_sleep', 'contributors.deep_sleep', 'contributors.rem_sleep', 'rem_sleep_duration', 'time_in_bed', 'bedtime_start_delta', 'bedtime_end_delta', 'bedtime_end_seconds', 'bed_time'], axis=1)

In [25]:
models_sleep_duration = lazy_regressor(X_total_sleep, y_total_sleep, test_size = 0.2, random_state = 89, regressor = reg)

print(models_sleep_duration)

'tuple' object has no attribute '__name__'
Invalid Regressor(s)


 65%|██████▌   | 26/40 [00:00<00:00, 35.36it/s]

LassoLarsIC model failed to execute
You are using LassoLarsIC in the case where the number of samples is smaller than the number of features. In this setting, getting a good estimate for the variance of the noise is not possible. Provide an estimate of the noise variance in the constructor.


100%|██████████| 40/40 [00:01<00:00, 35.86it/s]

RANSACRegressor model failed to execute
`min_samples` may not be larger than number of samples: n_samples = 8.
                               Adjusted R-Squared  R-Squared     RMSE  \
Model                                                                   
GaussianProcessRegressor                     7.61    -454.88 19536.44   
LinearSVR                                    7.59    -453.76 19512.47   
MLPRegressor                                 7.55    -450.61 19444.79   
HistGradientBoostingRegressor                2.02     -69.08  7659.85   
DummyRegressor                               2.02     -69.08  7659.85   
NuSVR                                        1.95     -64.52  7406.57   
ExtraTreesRegressor                          1.90     -61.09  7209.85   
RandomForestRegressor                        1.79     -53.23  6738.16   
QuantileRegressor                            1.75     -50.69  6578.32   
AdaBoostRegressor                            1.74     -49.96  6531.60   
SVR          

### LazyRegressor for the factors influencing the total sleep duration given the questionnaire variables only

In [26]:
X_total_sleep = df_quest[df_quest['Naturally'] == 1]

In [27]:
models_sleep_duration = lazy_regressor(X_total_sleep, y_total_sleep, test_size = 0.2, random_state = 89, regressor = reg)

print(models_sleep_duration)

'tuple' object has no attribute '__name__'
Invalid Regressor(s)


 75%|███████▌  | 30/40 [00:00<00:00, 60.35it/s]

LassoLarsIC model failed to execute
You are using LassoLarsIC in the case where the number of samples is smaller than the number of features. In this setting, getting a good estimate for the variance of the noise is not possible. Provide an estimate of the noise variance in the constructor.
RANSACRegressor model failed to execute
`min_samples` may not be larger than number of samples: n_samples = 8.


100%|██████████| 40/40 [00:00<00:00, 60.41it/s]

                               Adjusted R-Squared  R-Squared     RMSE  \
Model                                                                   
KernelRidge                                 21.05    -660.51 23533.72   
GaussianProcessRegressor                    14.81    -454.87 19536.26   
LinearSVR                                   14.80    -454.51 19528.45   
MLPRegressor                                14.78    -453.64 19509.97   
PassiveAggressiveRegressor                   5.83    -158.37 11551.30   
Lars                                         5.26    -139.42 10842.81   
HistGradientBoostingRegressor                3.12     -69.08  7659.85   
DummyRegressor                               3.12     -69.08  7659.85   
NuSVR                                        2.99     -64.52  7406.30   
QuantileRegressor                            2.59     -51.43  6625.44   
SVR                                          2.54     -49.92  6529.00   
KNeighborsRegressor                          2.42  

### LazyRegressor for the factors influencing the total sleep duration given the Oura variables only

In [28]:
X_total_sleep = df[df['Naturally'] == 1]
X_total_sleep = X_total_sleep.iloc[:, 0:49]
X_total_sleep = X_total_sleep.drop(labels=['total_sleep_duration', 'deep_sleep_duration', 'light_sleep_duration', 'contributors.total_sleep', 'contributors.deep_sleep', 'contributors.rem_sleep', 'rem_sleep_duration', 'time_in_bed', 'bedtime_start_delta', 'bedtime_end_delta', 'bedtime_end_seconds', 'bed_time'], axis=1)

In [29]:
models_sleep_duration = lazy_regressor(X_total_sleep, y_total_sleep, test_size = 0.2, random_state = 89, regressor = reg)

print(models_sleep_duration)

'tuple' object has no attribute '__name__'
Invalid Regressor(s)


 68%|██████▊   | 27/40 [00:00<00:00, 47.33it/s]

LassoLarsIC model failed to execute
You are using LassoLarsIC in the case where the number of samples is smaller than the number of features. In this setting, getting a good estimate for the variance of the noise is not possible. Provide an estimate of the noise variance in the constructor.
RANSACRegressor model failed to execute
`min_samples` may not be larger than number of samples: n_samples = 8.


100%|██████████| 40/40 [00:00<00:00, 41.13it/s]

                               Adjusted R-Squared  R-Squared     RMSE  \
Model                                                                   
GaussianProcessRegressor                    13.66    -454.88 19536.44   
LinearSVR                                   13.63    -453.76 19512.47   
MLPRegressor                                13.55    -450.95 19452.07   
DecisionTreeRegressor                        5.47    -160.08 11612.77   
BaggingRegressor                             3.16     -76.82  8071.92   
DummyRegressor                               2.95     -69.08  7659.85   
HistGradientBoostingRegressor                2.95     -69.08  7659.85   
NuSVR                                        2.82     -64.52  7406.67   
GradientBoostingRegressor                    2.81     -64.02  7378.15   
ExtraTreesRegressor                          2.71     -60.69  7186.68   
RandomForestRegressor                        2.66     -58.68  7068.35   
QuantileRegressor                            2.42  

### LazyRegressor for the factors influencing the total awake time during the night given all variables

In [30]:
# Separate the indenpendent variables from the dependent variables
# For the total awake time, Oura also considers some time before falling asleep and some time after waking up. 
# Subtract what we can from that, which is the time before falling asleep
y_awake_time = df['awake_time'] - df['latency']
X_awake_time = df.drop(labels=['awake_time', 'time_in_bed', 'latency', 'total_sleep_duration'], axis=1)

In [31]:
models_awake_time = lazy_regressor(X_awake_time, y_awake_time, test_size = 0.2, random_state = 56, regressor = reg)

print(models_awake_time)

'tuple' object has no attribute '__name__'
Invalid Regressor(s)


100%|██████████| 40/40 [00:02<00:00, 18.41it/s]

                               Adjusted R-Squared  R-Squared      RMSE  \
Model                                                                    
Lars                                     19455.89  -60411.55 202239.15   
GaussianProcessRegressor                     3.73      -7.48   2396.70   
MLPRegressor                                 3.58      -7.00   2326.79   
LinearSVR                                    3.26      -6.03   2181.61   
QuantileRegressor                            1.39      -0.23    911.21   
SVR                                          1.39      -0.22    910.34   
NuSVR                                        1.37      -0.15    881.45   
DummyRegressor                               1.36      -0.10    863.92   
KNeighborsRegressor                          1.30       0.06    796.90   
PoissonRegressor                             1.27       0.16    755.26   
GammaRegressor                               1.27       0.16    752.89   
TransformedTargetRegressor            

### LazyRegressor for the factors influencing the total awake time during the night given the questionnaire variables only

In [32]:
X_awake_time = df_quest

In [33]:
models_awake_time = lazy_regressor(X_awake_time, y_awake_time, test_size = 0.2, random_state = 56, regressor = reg)

print(models_awake_time)

'tuple' object has no attribute '__name__'
Invalid Regressor(s)


100%|██████████| 40/40 [00:01<00:00, 28.57it/s]

                               Adjusted R-Squared  R-Squared    RMSE  \
Model                                                                  
KernelRidge                                 13.14      -8.59 2547.45   
MLPRegressor                                11.54      -7.32 2373.72   
LinearSVR                                   11.11      -6.98 2324.97   
GaussianProcessRegressor                     8.80      -5.15 2041.32   
Lars                                         3.18      -0.72 1080.35   
ExtraTreeRegressor                           3.14      -0.69 1070.23   
TransformedTargetRegressor                   2.85      -0.46  995.56   
LinearRegression                             2.85      -0.46  995.56   
DecisionTreeRegressor                        2.85      -0.46  995.37   
HuberRegressor                               2.81      -0.43  984.33   
Lasso                                        2.80      -0.42  980.19   
RANSACRegressor                              2.77      -0.40  97

### LazyRegressor for the factors influencing the total awake time during the night given the Oura variables only

In [34]:
X_awake_time = df_oura.drop(labels=['awake_time', 'time_in_bed', 'latency', 'total_sleep_duration'], axis=1)

In [35]:
models_awake_time = lazy_regressor(X_awake_time, y_awake_time, test_size = 0.2, random_state = 56, regressor = reg)

print(models_awake_time)

'tuple' object has no attribute '__name__'
Invalid Regressor(s)


100%|██████████| 40/40 [00:01<00:00, 22.58it/s]

                               Adjusted R-Squared  R-Squared    RMSE  \
Model                                                                  
Lars                                        55.97     -77.12 7272.34   
GaussianProcessRegressor                     6.95      -7.46 2392.62   
MLPRegressor                                 6.64      -7.01 2329.32   
LinearSVR                                    5.95      -6.03 2181.61   
QuantileRegressor                            1.86      -0.23  911.21   
SVR                                          1.86      -0.22  910.24   
KNeighborsRegressor                          1.83      -0.18  894.67   
NuSVR                                        1.81      -0.15  881.05   
DummyRegressor                               1.78      -0.10  863.92   
ExtraTreeRegressor                           1.65       0.08  787.81   
GammaRegressor                               1.54       0.24  719.26   
TweedieRegressor                             1.44       0.37  65